### Requirements

In [ ]:
!pip install requests
!pip install json
!pip install pystac[orjson]
!pip install pystac[validation]
!pip install pystac
!pip install boto3
!pip install shapely
!pip install leafmap
!pip install ipyleaflet
!pip install awscli --user
!pip install opencv-python==3.4.8.29
!pip install flask


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


### Bibliotecas

In [ ]:
import pystac
import csv
import gzip
from io import StringIO
from urllib.request import urlopen
import requests
import os
import json
import random
import leafmap
from ipyleaflet import Map, GeoJSON
from PIL import Image
import cv2
import numpy as np
import zipfile



### Insira as varáveis para busca

In [ ]:
# Dicionário de câmeras do CBERS4

cameras = {
'AWFI' : 'https://s3.amazonaws.com/cbers-meta-pds/AWFIscene_list.csv.gz',
'MUX' : 'https://s3.amazonaws.com/cbers-meta-pds/MUXscene_list.csv.gz',
'PAN1' : ' https://s3.amazonaws.com/cbers-meta-pds/PAN10Mscene_list.csv.gz',
'PAN5M' : ' https://s3.amazonaws.com/cbers-meta-pds/PAN5Mscene_list.csv.gz'}

In [ ]:
#AWFI, MUX, PAN1 OU PAN5M
camera = cameras['MUX']

#Intervalo de datas para procurar
data_inicio = '2020-09-01'
data_fim = '2020-09-31'

#latitude e longitude
lat = -23.1791
lon = -45.8872



### Acessando AWS

#### Leitura e unzip dos links

In [ ]:
# Leitura e unzip
response = urlopen(camera)
gunzip_response = gzip.GzipFile(fileobj=response)
content = gunzip_response.read()
 
# Lendo o dicionário
scenes = list(csv.DictReader(StringIO(content.decode("utf-8"))))

In [ ]:
# Visualizando o formato do aqui

scenes[0]

#### Inserir filtros - Data e Local

##### Função que realiza o filtro

In [ ]:
 
def filtro_busca(scene):
    lat_max = float(scene['ul_lat'])
    lat_min = float(scene['lr_lat']) 
    lon_max = float(scene['ur_lon']) 
    lon_min = float(scene['ll_lon'])
    localizacao = lat_min <= lat and lat_max >= lat and \
                       lon_min <= lon and lon_max >= lon
    
    data_filtro = scene['acquisition_date'] >= '{}'.format(data_inicio) and scene['acquisition_date'] <= '{}'.format(data_fim)

    return data_filtro and localizacao
        


##### Realizando busca

In [ ]:
busca_satelite = [scene for scene in scenes if filtro_busca(scene)]


##### Resultados

In [ ]:
#Quantidade
len(busca_satelite)

In [ ]:
# Mostra as datas dos resultados

i = 0
resultado = []
if len(busca_satelite) > 1:
    while i < len(busca_satelite):
         busca = [busca_satelite[i]['acquisition_date'][0:10], busca_satelite[i]]
         resultado.append(busca)
         print(busca_satelite[0]['acquisition_date'][0:10])
         i = i + 1
else:
    busca = [busca_satelite[i]['acquisition_date'][0:10], busca_satelite[0]]
    resultado.append(busca)
    print(busca_satelite[0]['acquisition_date'][0:10])

In [ ]:
# Os resultados foram inseridos em uma lista e transformados em um dicionário 
resultado = dict(resultado)


In [ ]:
#Escolhe um dos resultados
data = '2020-09-24'
pesquisa = resultado[data]

In [ ]:
# Visualizando o resultado
pesquisa

### Acessando o STAC

In [ ]:
# Forma a URL de acesso ao STAC de acordo com o resultado dos filtros aplicados

url_json = "https://cbers-stac-1-0.s3.amazonaws.com/" + pesquisa['download_url']+".json"

In [ ]:
url_json

In [ ]:
#Acessa o STAC

link_json = requests.get(url_json)

# Faz a leitura do arquivo json

arquivo_json = link_json.json()

### Acessando, Visualizando e Baixando as Imagens

#### Imagens JPG

In [ ]:
#Função para visualizar a imagem em formato JPG

def visualizar_jpg(url):
    img_jpg = Image.open(requests.get(url, stream=True).raw)
    return img_jpg

In [ ]:
#Ação para visualizar a imagem

img_jpg_url = arquivo_json['assets']['thumbnail']['href']
visualizar_jpg(img_jpg_url)

#### Imagem TIFF

In [ ]:
#Filtro para acessar a imagem dentro do arquivo JSON

img_jpg_url = arquivo_json['assets']['thumbnail']['href']
img_jpg_url

In [ ]:
#Função para baixar imagem TIFF

def baixar_img_tiff(url):
  with open("imagem-satelite.tiff", 'wb') as imagem:
    resposta = requests.get(url, stream=True)

    if not resposta.ok:
      print("Ocorreu um erro, status:" , resposta.status_code)
    else:
      for dado in resposta.iter_content(1024):
        if not dado:
            break

        imagem.write(dado)

  return print("Imagem salva! É só acessar sua pasta agora =)")

In [ ]:
# Ação para baixar a imagem TIFF

baixar_img_tiff(img_jpg_url)

In [ ]:
#Função para baixar as bandas separadas e salvar em um zip

def baixar_bandas():
    
    # Separando a imagem em 3 bandas e salvando
    img = cv2.imread("imagem-satelite.tiff")
    (canalAzul, canalVerde, canalVermelho) = cv2.split(img)
    zeros = np.zeros(img.shape[:2], dtype = "uint8")


    # Código abaixo abre as imagens
    #cv2.imshow("Banda_4_vermelho", cv2.merge([zeros, zeros, canalVermelho]))
    #cv2.imshow("Banda_3_verde", cv2.merge([zeros, canalVerde, zeros]))
    #cv2.imshow("Banda_2_azul", cv2.merge([canalAzul, zeros, zeros]))
    #cv2.waitKey(0)


    # Salvando imagems fragmentadas em bandas RGB
    cv2.imwrite("Banda_4_vermelho.tiff", cv2.merge([zeros, zeros, canalVermelho]))
    cv2.imwrite("Banda_3_verde.tiff", cv2.merge([zeros, canalVerde, zeros]))
    cv2.imwrite("Banda_2_azul.tiff", cv2.merge([canalAzul, zeros, zeros]))

    
    # Abrindo uma pasta .zip, armazenando e fechando
    z = zipfile.ZipFile('satelite.zip', 'w', zipfile.ZIP_DEFLATED)
    z.write("Banda_4_vermelho.tiff")
    z.write("Banda_3_verde.tiff")
    z.write("Banda_2_azul.tiff")
    z.close()
    return print("Salvo!! É só acessar sua pasta agora =)")

In [ ]:
#Ação para baixar as bandas

baixar_bandas()

In [ ]:
def contraste_brilho():
    img = cv2.imread("imagem-satelite.tiff")

    alpha = 2 # Controle de Contraste (pode variar de 1.0 a 3.0)
    beta = 0 # Controle de brilho (pode variar de 0 a 100)

    adjuste = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    cv2.imwrite("novasatelite.tiff", adjuste)
    return print("Salvo!! É só acessar sua pasta agora =)")

#### LeafMap Dinâmico com as Imagens

##### Marcar local de abrangência

In [ ]:
#Função para gerar mapa com marcador 


def marcador_leafmap(data):
    m = leafmap.Map(center=(lat, lon),zoom=7)
    geo_json = GeoJSON(
        data=data)
    m.add_layer(geo_json)
    return m



In [ ]:
# Visualizar mapa

marcador_leafmap(arquivo_json)
